# Miniproyecto 3

### David Soto 17551 & Miguel Valle 17102

In [1]:
### Librerias a utilizar para la simulacion
import numpy as np
import random

### Función generadora de tiempos de estadía/salida

In [2]:
# Funcion para calcular el proximo tiempo de llegada
def tiempoProximaSalida(t, lamda_max, homo):
    if homo:
        return t - ((1/lamda_max)*np.log(random.random()))
    else:
        while True:
            u = random.random()
            t = t - ((1/lamda_max)*np.log(u))
            u = random.random()
            if u <= np.random.poisson(t)/lamda_max:
                return t

### Funcion simuladora de eventos

In [3]:
### Funcion que determina cómo procede el evento segun el tiempo dela simulacion
def evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed):
    ### Si la proxima peticion es el ultimo registrado en el sistema, entonces hay un servidor libre que lo puede atender
    if ta == min(ta,min(td)): 
        t = ta
        Na = Na + 1
        ta = tiempoProximaSalida(t, lamda_max, True)
        A[Na] = t
        ### Si la cantidad de peticiones en el sistema es menor a la cantidad de cpus
        ### es porque la proxima peticion puede ser procesada por un servidor
        if n < no_cpus:
            for i in range(no_cpus):
                if ids[i] == 0:
                    ids[i] = Na
                    NT = np.append(NT, t - A[Na])
                    td[i] = t + np.random.exponential(speed)
                    busyTime[i] = busyTime[i] + td[i] - t
                    break
        n = n + 1
    else:
        ### Esto ocurre en caso de que no haya un servidor desocupado al momento de llegar
        ti = np.argmin(td)
        t = td[ti]
        C[ti] = C[ti] + 1
        D[ids[ti]] = t
        ### En caso la cantidad de clientes sea menor o igual a la cantidad de servidores
        ### Se libera un CPU
        if n <= no_cpus:
            ### Se elimina a la ultima peticion del ID register
            ids[ti] = 0
            td[ti] = np.infty
        ### Si todavia hay mas clientes por atender
        else: 
            indice = max(ids) + 1
            ### Se actualiza el indice
            ids[ti] = indice
            NT = np.append(NT , t - A[indice])
            td[ti] = t + np.random.exponential(speed)
            busyTime[ti] = busyTime[ti] + td[ti] - t
        n = n - 1
        
    ### Retornamos las variables para actualizacion y seguir el proceso de la simulacion 
    return t, n, Na, C, busyTime, A, D, NT, ta, td, ids

### Funcion simuladora del procesamiento de peticiones

In [4]:
### Funcion para realizar la simulacion
def simular(duracion_simulacion, lamda_max, speed,no_cpus):
    ### Variable de tiempo de simulacion
    t = 0
    
    # Variables contadores
    Na = 0
    C = np.zeros(no_cpus) 
    busyTime = np.zeros(no_cpus)
    
    ### Variable estado del sistema
    n = 0

    ### Variables para el control de datos
    A = {}
    D = {}
    NT = []
    
    # Variables de eventos de tiempos de llegada y salida
    ta = tiempoProximaSalida(t,lamda_max, True)
    td = np.zeros(no_cpus) + np.infty
    ids = np.zeros(no_cpus)
    
    # Se realiza la simulacion hasta que se llegue al tiempo final de esta
    while t < duracion_simulacion:
        ### Realizamos la simulacion de un evento al timepo t
        t, n, Na, C, busyTime, A, D, NT, ta, td, ids = evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed)
        
    ### Obtenemos el tiempo de salida de la ultima solicitud atendida
    copiaArreglo = td
    tiemposSalida = []
    for i in copiaArreglo:
        if not np.isinf(i):
            tiemposSalida.append(i)

    ### Obtenemos los segundos promedio por solicitud
    copiaArreglo = NT
    segundoSolicitud = []
    cantidadSolicitudes = 0
    for i in copiaArreglo:
        if i != 0:
            segundoSolicitud.append(1/i)
        cantidadSolicitudes = cantidadSolicitudes + 1
    

    ### Mostramos los resultados de las variables de la somulacion
    print("Solicitudes atendidas: ", C)
    print("Tiempos ocupado de servidores: ", busyTime)
    print("Tiempos desocupado o IDLE de servidores: ", np.maximum(np.ones(no_cpus)*duracion_simulacion - busyTime,0))
    print("Tiempo total de solicitudes en cola: ", np.round(sum(NT),5))
    print("Promedio de solicitudes por segunda en cola:", np.round(np.mean(NT),5))
    print("Promedio de segundos por solicitud en cola:", np.round(sum(segundoSolicitud)/cantidadSolicitudes,5))
    print("Tiempo de salida de la ultima solicitud atendida: ", np.round(max(tiemposSalida),8))

## Task 1 - Simulacion con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto -  Gorilla Megacomputing

In [6]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [144150.]
Tiempos ocupado de servidores:  [24.1571796]
Tiempos desocupado o IDLE de servidores:  [35.8428204]
Tiempo total de solicitudes en cola:  16.17947
Promedio de solicitudes por segunda en cola: 0.00011
Promedio de segundos por solicitud en cola: 13801.87958
Tiempo de salida de la ultima solicitud atendida:  60.00080362


### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [7]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [28711. 26773. 23874. 20320. 16214. 11918.  7834.  4620.  2510.  1252.]
Tiempos ocupado de servidores:  [48.16250509 44.31704777 39.55490468 33.42812442 26.85719644 19.72347513
 13.05597708  7.75081692  4.16618087  2.06325442]
Tiempos desocupado o IDLE de servidores:  [11.83749491 15.68295223 20.44509532 26.57187558 33.14280356 40.27652487
 46.94402292 52.24918308 55.83381913 57.93674558]
Tiempo total de solicitudes en cola:  0.29525
Promedio de solicitudes por segunda en cola: 0.0
Promedio de segundos por solicitud en cola: 272.69906
Tiempo de salida de la ultima solicitud atendida:  60.0029652


## Task 2 - Determinacion de numero de servidores para no tener cola con Ants con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [8]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 15

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [2.9000e+04 2.6537e+04 2.3638e+04 2.0251e+04 1.6120e+04 1.1873e+04
 7.9370e+03 4.7780e+03 2.4510e+03 1.1850e+03 4.7900e+02 1.9600e+02
 6.3000e+01 2.1000e+01 3.0000e+00]
Tiempos ocupado de servidores:  [4.79220307e+01 4.43281649e+01 3.96537521e+01 3.35825270e+01
 2.68578359e+01 1.97336986e+01 1.33055234e+01 7.94158171e+00
 4.06200498e+00 1.93808579e+00 8.77698183e-01 3.15991279e-01
 1.24972875e-01 3.64303369e-02 8.27687978e-03]
Tiempos desocupado o IDLE de servidores:  [12.07796928 15.67183509 20.34624789 26.41747304 33.14216409 40.26630143
 46.69447656 52.05841829 55.93799502 58.06191421 59.12230182 59.68400872
 59.87502712 59.96356966 59.99172312]
Tiempo total de solicitudes en cola:  0.0
Promedio de solicitudes por segunda en cola: 0.0
Promedio de segundos por solicitud en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  60.00465181


## Task 3 - Simulacion con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Gorilla Megacomputing

In [9]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [358945.]
Tiempos ocupado de servidores:  [59.80776491]
Tiempos desocupado o IDLE de servidores:  [0.19223509]
Tiempo total de solicitudes en cola:  24982.55333
Promedio de solicitudes por segunda en cola: 0.0696
Promedio de segundos por solicitud en cola: 213.91005
Tiempo de salida de la ultima solicitud atendida:  60.0001244


### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [10]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [35811. 35897. 35669. 36031. 36180. 35943. 36063. 35863. 35868. 35873.]
Tiempos ocupado de servidores:  [59.9821015  59.97730015 59.97630106 59.96313795 59.96482392 59.9461379
 59.94527722 59.93105132 59.9233265  59.89553427]
Tiempos desocupado o IDLE de servidores:  [0.0178985  0.02269985 0.02369894 0.03686205 0.03517608 0.0538621
 0.05472278 0.06894868 0.0766735  0.10446573]
Tiempo total de solicitudes en cola:  42284.06297
Promedio de solicitudes por segunda en cola: 0.11771
Promedio de segundos por solicitud en cola: 67.06999
Tiempo de salida de la ultima solicitud atendida:  60.00899053


## Task 4 - Determinacion de numero de servidores para no tener cola con Ants con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [11]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 30

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [3.2471e+04 3.2187e+04 3.1388e+04 3.0802e+04 2.9726e+04 2.8663e+04
 2.7374e+04 2.5430e+04 2.3506e+04 2.1115e+04 1.8513e+04 1.5921e+04
 1.2825e+04 9.9720e+03 7.2410e+03 5.1640e+03 3.4340e+03 2.0690e+03
 1.2600e+03 6.8100e+02 3.5500e+02 1.6500e+02 5.6000e+01 1.9000e+01
 8.0000e+00 2.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
Tiempos ocupado de servidores:  [5.46226503e+01 5.36444478e+01 5.25407519e+01 5.12731626e+01
 4.96964115e+01 4.76803601e+01 4.54711589e+01 4.25531076e+01
 3.91959483e+01 3.52054587e+01 3.10408046e+01 2.62572636e+01
 2.14880274e+01 1.67694645e+01 1.22292925e+01 8.41657290e+00
 5.82213749e+00 3.62793862e+00 2.02683990e+00 1.11075583e+00
 5.34585833e-01 2.82970821e-01 1.07716224e-01 3.64045490e-02
 1.13301272e-02 3.15641459e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Tiempos desocupado o IDLE de servidores:  [ 5.37734973  6.35

## Task 5 - Recomendacion a la junta directiva

Dado que se observa una mejora clara en el tiempo de espera para las solicitudes hechas, se recomienda alquilar 15 servidores para un numero de solicitudes de 2400, y de alquilar 30 servidores para 6000 solicitudes, ya que aunque alquilar más servidores podría reducir aún más el tiempo de espera, no se estaran utilizando todos cuando no se tenga una cantidad de solicitudes muy elevada, por lo cual sería un desperdicio del presupuesto. Es preferible tener menos servidores, pero que todos se estén aprovechando durante un flujo normal de solicitudes realizada, tomando como línea base 2400 y límite superior valores cercanos a 6000. Por lo tanto la cantidad de servidores que se recomienda alquilar se encuentra en un rango entre 15 y 30 servidores.